<a href="https://colab.research.google.com/github/mckang6113875/AP_Test_OPEN/blob/main/Temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 6113875

https://blog.naver.com/PostView.nhn?blogId=chai1226&logNo=221944798109&parentCategoryNo=&categoryNo=&viewDate=&isShowPopularPosts=false&from=postView

https://github.com/alexvbogdan/DeepCalib.git

https://colab.research.google.com/github/alexvbogdan/DeepCalib.git

In [11]:
! pip install tensorflow-graphics

     |████████████████████████████████| 4.3 MB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 46.0 MB/s 
     |████████████████████████████████| 646 kB 55.5 MB/s 
     |████████████████████████████████| 280 kB 41.6 MB/s 
     |████████████████████████████████| 462 kB 39.6 MB/s 
  Created wheel for OpenEXR: filename=OpenEXR-1.3.8-cp37-cp37m-linux_x86_64.whl size=189479 sha256=ea8af8de3a30702f75c9867b44ff836d976bf7932e27b0e0efd5c8b44c599b72
  Stored in directory: /root/.cache/pip/wheels/c9/2d/94/33b42ddc382c59626bbb75ad86575339decee5880dbbf7b182
Successfully built OpenEXR
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [8]:
#
from google.colab import files
src = list(files.upload().values())[0]

Saving .bazelrc to .bazelrc
Saving .flake8 to .flake8
Saving .gitignore to .gitignore
Saving .gitmodules to .gitmodules
Saving .pylintrc to .pylintrc
Saving CONTRIBUTING.md to CONTRIBUTING.md
Saving LICENSE to LICENSE
Saving MANIFEST.in to MANIFEST.in
Saving pytest.ini to pytest.ini
Saving README.md to README.md
Saving requirements.txt to requirements.txt
Saving requirements.unix to requirements.unix
Saving setup.py to setup.py
Saving WORKSPACE to WORKSPACE


In [9]:
#
from google.colab import files
src = list(files.upload().values())[0]

Saving __init__.py to __init__.py
Saving orthographic.py to orthographic.py
Saving perspective.py to perspective.py
Saving quadratic_radial_distortion.py to quadratic_radial_distortion.py


In [13]:
import quadratic_radial_distortion

In [14]:
import perspective